# TABLE OF CONTENT
+ THE BUSINESS CASE
+ THE DATA
+ THE METHODOLOGY

# THE BUSINESS PROBLEM
According to [Wikipedia](https://en.wikipedia.org/wiki/Types_of_restaurants), you can classify restaurants into seven categories that in turn can be broken down into eleven variations. Some of these combinations might be a little odd (e.g. "Fine Dining" and "Pub") but let us assume for a minute that this list is complete and all combinations are possible. In result, we would have 77 unique combinations of how a restaurant could look like. [Wikipedia](https://en.wikipedia.org/wiki/List_of_cuisines) also hosts a list of cuisines and cuisine types. According to Wikipedia, there are five major cusine styles and 123 distinguished ethnic and religious cuisines, giving us 615  combinations for different types of food that we could eat. If we trust these figures, we are all in all looking at a possible variety of over 43,000 unique restaurants - great, right?
<br>
<br>
![But why](https://media.giphy.com/media/s239QJIh56sRW/giphy.gif)
<br>
<br>
Glad you asked! These figures do not really matter for our problem. I only wanted to show you one thing: The food industry is extremely diverse and each type of restaurant brings its own unique challenges. However, they all have one common issue: Location, location, location! No matter what type of restaurant you own, you need a specific location that matches your profile. Of course you could now say: "That's easy to fix - just go where a lot of people are!" 
<br>
<br>
But is it really so easy? Depending on your restaurant, you need parking space. Maybe you want a quite neighborhood because you own a fine dining restaurant. Maybe you don't want to be where a lot of other restaurants are that are similiar to the cuisine you offer. Or maybe you have a super trendy concept that requires your restaurant to be placed in the most hipster district of your city. We could make this list longer and longer but the point is: It ain't easy. 
<br>
<br>
Choosing the right location also becomes increasingly difficult when you don't know the city you want to open your restaurant in. Finding the right spot will require you to do some extensive research and location scouting. This process takes a lot of time and eats up ressources. While this approach is feasible for small businesses, it becomes unhandy for larger corporations who want to expand their restaurants to multiple cities at the same time. In result, corporations need to spend more money to keep up quality and speed. So how could we help larger restaurant corporations to make this process more efficient? 
<br>
<br>
Well, with data science of course! Larger restaurant chains usually already have a couple of restaurants in place and know which locations run well in their business model and which not. We can use this information to train a clustering algorithm to identify neighborhoods that are similar to the locations with profitable restaurants. This way we can build a recommender system that tells us which neighborhoods in a new town are more likely to host a successful restaurant. Consequently, we would not have to research all areas but could focus our scouting activity on the most promising neighborhoods. This way, we would be able to improve speed and quality as well as cut the related costs significantly.

<br>
<br>

# THE DATA

## L'OSTERIA
As our business problem aims to build a recommender system for a fast growing restaurant chain, we need an example that we can work with. Ladies and gentlemen, please welcome my favorite pizza chain: [L'Osteria](https://losteria.net/en/)!
<br>
<br>
L'Osteria is an italian restaurant chain that was founded in Germany in 1999. The first restaurant was quickly becomming a hit as it combined tasty food, a trendy atmosphere and reasonable prices. Today, the chain has expanded to basically every larger city in Germany and starts to go international with places in Switzerland, Austria, Netherlands and the UK.
<br>
<br>
The chain has a major focus on Germany and it basically has one (or even multiple) restaurants in every bigger city in Germany. That gives us a lot of possibilities to train our algorithm. Unfortunately, we do not have access to L'Osteria's database which is why we need to construct our own workaround. Luckily for us, the chain publishes the addresses of all their restaurants on their [website](https://losteria.net/en/restaurants/view/list/?tx_losteriarestaurants_restaurantsreopening%5Bfilter%5D%5Bcountry%5D=de&tx_losteriarestaurants_restaurantsreopening%5Bfilter%5D%5BsearchTerm%5D=&&&tx_losteriarestaurants_restaurantsreopening%5Bfilter%5D%5Btype%5D=reservations&).
<br>
<br>
To use this information, we need to scrape it from their website. When you take a closer look at the L'Osteria website, you will notice that it uses an event button which hides most of the restaurants. We could use a library called [Selenium](https://www.selenium.dev/), if we want a fully automized script that clicks the button for us. However, that would be an overkill as the button only needs to be clicked ~10 times. In result, it is easier to do so by hand and download the website that now shows all restaurants and store it as a html file in our working directory. 
<br>
<br>
That enables us to use the infamous html scraping library [BeautifulSoup](https://www.crummy.com/software/BeautifulSoup/bs4/doc/). The following code scrapes the data we need from the downloaded html file (also available in the repository) and transforms it as the file contains e.g German characters that otherwise result in jibberish. 

In [4]:
# Import working libraries
import pandas as pd # Data wrangling
from tqdm import tqdm # Progress visualization
import requests # Request handling
from bs4 import BeautifulSoup # HTML search
import re # String handling
import ast # String handling

# Set path to html file
path = r"C:\Users\maurice.buettgenbach\OneDrive - Aquila Capital Management GmbH\Desktop\Desktop\Private\IBM\10_Capstone project\IBM_The_battle_of_neighborhoods_v2\LOSTERIA_DATA\Restaurants - L'Osteria.htm"

# Open document and transform to BS object
with open(path) as page:
    soup = BeautifulSoup(page, 'html.parser')

# Define results
results = soup.find(id='losteria-restaurants-list-wrapper')

# Find elements
page_elements = results.find_all('div', class_='address')

# Create variable to store data in
restaurants = pd.DataFrame()

# Iterate through found elements
for page_element in tqdm(page_elements):
    # Isolate address
    element = str(page_element)
    # Get rid of unnecessary characters
    element = element.replace("\n", "")
    element = element.replace("\r", "")
    # Get rid of spaces
    element = element.replace(" ", "")
    # Search for address in between <div>
    search_result = re.search(">(.*)<", element)
    # Store search result in new variable
    address = search_result.group(1)
    # Get street
    street = address.split(",")[0]
    # Add space between street name and house number
    street = re.sub(r"([0-9]+(\.[0-9]+)?)",r" \1 ", street).strip()
    # Add space before Capital letter
    street = re.sub(r"(?<=\w)([A-Z])", r" \1", street).strip()
    # Get ZIP code and city
    address = address.split(",")[1]
    # Isolate ZIP code
    zip_code = re.split('(\d+)', address)[1]
    # Isolate city
    city = re.split('(\d+)', address)[2]
    # Create temporary df
    temp_df = pd.DataFrame()
    # Add data to df
    temp_df['STREET'] = [street]
    temp_df['ZIP_CODE'] = zip_code
    temp_df['CITY'] = city
    # Append to variable
    restaurants = restaurants.append(temp_df)
# Reset index
restaurants.reset_index(drop=True, inplace=True)

# Replace cryptic html for German characters
restaurants = restaurants.replace('Ã¤', "ae", regex=True) # For German Umlaut "ä"
restaurants = restaurants.replace('Ã¼', "ue", regex=True) # For German Umlaut "ü"
restaurants = restaurants.replace('Ã¶', "oe", regex=True) # For German Umlaut "ö"
restaurants = restaurants.replace('ÃŸ', "ss", regex=True) # For German Umlaut "ß"

# Add spacer for output
print()
print()

# Visualize variable
print("df shape:", restaurants.shape)
restaurants.head(10)

100%|██████████| 121/121 [00:00<00:00, 438.69it/s]




df shape: (121, 3)


,STREET,ZIP_CODE,CITY
0,Gut-Daemme-Strasse 1,52070,Aachen
1,Franziskanergasse 5,92224,Amberg
2,Pearl-S.-Buck-Strasse 12,86156,Augsburg
3,Albert-Schenavsky-Str. 2,86165,Augsburg
4,Maximilianstrasse 83,95444,Bayreuth
5,Hilda-Geiringer-Weg 4,10557,Berlin
6,Alt Mahlsdorf 88,12623,Berlin
7,Memhardstrasse 3,10178,Berlin
8,Mildred-Harnack-Str. 11,10243,Berlin
9,Savignyplatz 5,10623,Berlin


## UBER H3 
I was lucky enough to travel a lot in my youth but it is still impossible for me to tell whether a restaurant is located in a trendy neighborhood or not. To draw any conclusions on the neighborhoods the restaurants are located in, I would have to google their locations and research the respective city as a whole. However, L'Osteria has over 100 restaurants in Europe - it would therefore take me days or even weeks to conduct a respective analysis. 
<br>
<br>
Since this is anything else but ressource efficient, we have to find a way of speeding things up. In order to analyse the location of each restaurant, we need to define a radius from which we want to draw conclusions from. One approach would be to get shapefiles for each neighborhood in each city L'Osteria has a restaurant in and then download information related to these neigborhoods. But good luck trying to get these shapefile for Germany. Here, every state has its own data service - one more confusing than the other. Collecting the necessary data would take weeks and provide us with a minimal granularity.
<br>
<br>
This is where Uber's H3 hexagonial mapping framework comes in play. At its core, H3 is a geospatial analysis tool that provides a global index of hirachically ordered hexagons. Just imagine that you would map the entire earth (pole to pole) in hexagonial polygons. Now, each hexagon would receive a unique ID so that you could exactly tell what place on earth belongs to which hexagon. And finally, think of different layers of hexagons. Each layer is much smaller and more precise, enabling you to drill up and down on your map. That is exactly what H3 does - it provides you unique hexagons with different resolutions, all the way down to a precision of 0.0000009 km² area. 
<br>
<br>
So how is this helping us? By knowing the GPS coordinates of the L'Osteria restaurants, we can assign them to an unique hexagon. This hexagon will then serve as our designated radius for which we will download further data for our analysis. The H3 framework also helps us to map any city on earth with the same size of hexagons, making it easier for us to compare our results. 

## OPEN STREET MAP
We can analyze the area of our L'Osteria locations by downloading the information linked to the neighbouring hexagons by defining the degree of neighbours we want to take into account. If we analyze all first degree neighbours, we would analyze the information of the six hexagons that are directly linked to the hexagon a restaurant is located in. And if we would analyze the second degree neighbours, we would analyze all hexagons that are linked to the first degree neighbours as well. 
<br>
<br>
However, if we want to analyze a specific area such as a city, we cannot use this approach since we are limited to borders that are not considered under H3. The closer we get to the borders of a city, the more hexagons we would observe that are located outside of the city's borders. Hence, our data could be distorted. So what do we do?
<br>
<br>
We need shape files. A shape file is a specific data format that allows you to handle geospatial data. In our case, we are specifically interested in shapefiles that contain border information of European cities. There are many data providers that offer this information in form of a shape file. Since we are on a budget, we will use the shape files provided by [OpenStreetMap](https://www.openstreetmap.de/). OpenStreetMap is an international project founded in 2004 with the goal of creating a free map of the world. For this purpose they collect data about roads, railroads, rivers, forests, houses and much more worldwide. However, the shapefiles we are interested in are provided by [Geofabrik](https://www.geofabrik.de/) which is a German provider who built its service based on OpenStreetMap. Here, you can download shapefiles of various countries, regions and cities for free.

## FOURSQUARE
[Foursquare](https://foursquare.com/) is an app that lets its users rank all different kinds of venues all over the world. By doing so, Foursquare has build one of the most comprehensive geospatial data sets in the world. The data set covers an extremely large amount of venue and user data that allows you to access everything from a venue's name, location or menu up to user ratings, comments or pictures of each venue. And the best part: Foursquare provides developers an [API](https://developer.foursquare.com/) that is (in limits) free to use! 
<br>
<br>
Even though Foursquare provides an incredible variety of data, we will use the venue categories (e.g. sushi restaurant, park, sports club, italian restaurant, etc.) for our analysis only. While it would most likely increase the success of our algorithm to include more information such as rankings or types of user data, we will not include this data due to ressource limitations.
<br>
<br>
We will use the Foursquare data to analyze the area of the hexagons (or neighbouring hexagons) in which L'Osteria has existing restaurants as well as the hexagons of the new city we are interested in. This way we will be able to cluster the hexagons according to their characteristics.
<br>
<br>